# Model Optimization & Hyperparameter Tuning

## Purpose
This notebook focuses on optimizing the best-performing sequence model from previous experiments (GRU) to further improve sentiment classification accuracy.  
We will adjust key hyperparameters, experiment with deeper architectures, and evaluate the impact of these changes on validation accuracy.

## Steps
1. Load preprocessed IMDb dataset.
2. Define a model-building function compatible with hyperparameter tuning.
3. Use KerasTuner to search for optimal hyperparameters:
   - Number of GRU units
   - Number of layers
   - Dropout rates
   - Learning rate
   - Batch size
4. Train the best-found model and compare it with the previous version.


In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import kerastuner as kt

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25080\2424917221.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [12]:
# Load Data
X_train = pd.read_csv(r"F:\Projects\Sentiment Analysis\data\X_train.csv")['clean_review']
X_test = pd.read_csv(r"F:\Projects\Sentiment Analysis\data\X_test.csv")['clean_review']
y_train = pd.read_csv(r"F:\Projects\Sentiment Analysis\data\y_train.csv").squeeze()
y_test = pd.read_csv(r"F:\Projects\Sentiment Analysis\data\y_test.csv").squeeze()

In [13]:
# Tokenization
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [14]:
# Model builder for KerasTuner
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
    
    # Number of GRU units
    gru_units = hp.Int('gru_units', min_value=32, max_value=128, step=32)
    model.add(GRU(gru_units, return_sequences=False))
    
    # Dropout rate
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.6, step=0.1)
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Learning rate
    lr = hp.Choice('learning_rate', values=[1e-4, 1e-3, 1e-2])
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [15]:
# Initialize tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='gru_tuning'
)

f:\Projects\Sentiment Analysis\imdb-sentiment-rnn-env\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
# Run tuner
tuner.search(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test))

Trial 5 Complete [00h 08m 56s]
val_accuracy: 0.7690833806991577

Best val_accuracy So Far: 0.8845416903495789
Total elapsed time: 00h 52m 41s


In [17]:
# Get best model
best_hp = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test))

Epoch 1/5
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 110s 87ms/step - accuracy: 0.5030 - loss: 0.6939 - val_accuracy: 0.5197 - val_loss: 0.6905
Epoch 2/5
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 108s 87ms/step - accuracy: 0.5432 - loss: 0.6832 - val_accuracy: 0.8191 - val_loss: 0.4303
Epoch 3/5
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 111s 89ms/step - accuracy: 0.8670 - loss: 0.3313 - val_accuracy: 0.8849 - val_loss: 0.2976
Epoch 4/5
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 108s 87ms/step - accuracy: 0.9297 - loss: 0.1936 - val_accuracy: 0.8843 - val_loss: 0.3345
Epoch 5/5
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 109s 88ms/step - accuracy: 0.9612 - loss: 0.1237 - val_accuracy: 0.8787 - val_loss: 0.3918


In [18]:
best_model.save(r"F:\Projects\Sentiment Analysis\models\gru_tuned_model.h5")

In [19]:
import joblib
joblib.dump(tokenizer, r"F:\Projects\Sentiment Analysis\models\tokenizer.pkl")

['F:\\Projects\\Sentiment Analysis\\models\\tokenizer.pkl']

In [20]:
import json

results = {
    "model": "GRU Tuned",
    "best_hyperparameters": {
        "gru_units": best_hp.get('gru_units'),
        "dropout_rate": best_hp.get('dropout_rate'),
        "learning_rate": best_hp.get('learning_rate')
    },
    "validation_accuracy": float(best_model.evaluate(X_test_pad, y_test)[1])
}

with open(r"F:\Projects\Sentiment Analysis\models\gru_tuned_results.json", "w") as f:
    json.dump(results, f, indent=4)


310/310 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8789 - loss: 0.3917
